In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.font_manager as fm

plt.rcParams['font.family'] = 'Malgun Gothic' 
plt.rcParams['axes.unicode_minus'] = False

In [5]:
multi_household = pd.read_csv("C:/Users/rmagm/OneDrive/바탕 화면/vscode/SDC_project/multi_household/단독다가구23,24합본.csv")

In [6]:
multi_household['계약연월'] = multi_household['계약년월'].astype(str)  # 예: 202412

# 주소 + 계약연월 기준 그룹화
grouped = multi_household.groupby(['도로명', '계약연월']).size().reset_index(name='동일주소_해당월_거래수')

In [7]:
# 면적대, 건축연도대 생성
multi_household['면적대'] = pd.cut(multi_household['계약면적(㎡)'], bins=[0, 30, 50, 70, 100, 200], labels=['0-30', '30-50', '50-70', '70-100', '100+'])
multi_household['건축연도대'] = pd.cut(multi_household['건축년도'], bins=[0, 1980, 1990, 2000, 2010, 2020, 2030], labels=['~80', '80-90', '90-00', '00-10', '10-20', '20+'])

# 조건 그룹 생성
multi_household['조건그룹'] = multi_household[['구', '건축연도대', '면적대']].astype(str).agg('_'.join, axis=1)


In [8]:
# 그룹별 평균과 표준편차 → Z-score
group_stats = multi_household.groupby('조건그룹')['보증금(만원)'].agg(['mean', 'std']).reset_index()
group_stats.columns = ['조건그룹', '보증금_평균', '보증금_표준편차']
multi_household = multi_household.merge(group_stats, on='조건그룹', how='left')
multi_household['보증금_zscore'] = (multi_household['보증금(만원)'] - multi_household['보증금_평균']) / multi_household['보증금_표준편차']
multi_household['보증금_이상치'] = multi_household['보증금_zscore'].abs() > 2

In [9]:
multi_household['도로명_연월'] = multi_household['도로명'] + '_' + multi_household['계약연월'].astype(str)
도로_빈도 = multi_household['도로명_연월'].value_counts().reset_index()
도로_빈도.columns = ['도로명_연월', '해당월_거래수']
multi_household = multi_household.merge(도로_빈도, on='도로명_연월', how='left')
multi_household['거래_집중의심'] = multi_household['해당월_거래수'] >= 3

In [10]:
multi_household['노후건물'] = multi_household['건축년도'] < 1990
multi_household['노후_이상보증'] = multi_household['노후건물'] & multi_household['보증금_이상치']

In [11]:
구별_지표 = multi_household.groupby('구').agg({
    '보증금_이상치': 'sum',
    '거래_집중의심': 'sum',
    '노후_이상보증': 'sum',
    '도로명': 'count'  # 총 거래 수로 사용
}).reset_index()

구별_지표['보증금_이상비율'] = 구별_지표['보증금_이상치'] / 구별_지표['도로명']
구별_지표['거래집중비율'] = 구별_지표['거래_집중의심'] / 구별_지표['도로명']
구별_지표['노후이상비율'] = 구별_지표['노후_이상보증'] / 구별_지표['도로명']

In [12]:
from sklearn.preprocessing import RobustScaler  # 또는 StandardScaler, MinMaxScaler

scaler = RobustScaler()
score_cols = ['보증금_이상비율', '거래집중비율', '노후이상비율']
구별_지표[[f'{col}_정규화' for col in score_cols]] = scaler.fit_transform(구별_지표[score_cols])

# 종합 스코어
구별_지표['위험스코어'] = 구별_지표[[f'{col}_정규화' for col in score_cols]].mean(axis=1)

# 상위 5개 구 추출
상위구 = 구별_지표.sort_values('위험스코어', ascending=False).head(5)

In [13]:
상위구

,구,보증금_이상치,거래_집중의심,노후_이상보증,도로명,보증금_이상비율,거래집중비율,노후이상비율,보증금_이상비율_정규화,거래집중비율_정규화,노후이상비율_정규화,위험스코어
4,관악구,306,1747,26,5497,0.055667,0.317810,0.004730,2.391325,1.706724,-1.241303,0.952249
1,강동구,126,648,43,2621,0.048073,0.247234,0.016406,0.803946,0.731870,1.025091,0.853636
11,동작구,185,1128,30,3736,0.049518,0.301927,0.008030,1.106005,1.487341,-0.600730,0.664205
10,동대문구,118,453,30,2328,0.050687,0.194588,0.012887,1.350396,0.004675,0.341966,0.565679
5,광진구,193,1229,41,4314,0.044738,0.284886,0.009504,0.106740,1.251959,-0.314626,0.348024


In [ ]:
from sklearn.preprocessing import MinMaxScaler

def analyze_risk(df):
    # -------------------
    # 1. 전처리
    # -------------------
    df = df[['구', '계약면적(㎡)', '보증금(만원)', '계약년월', '계약일', '건축년도', '계약구분', '도로명', '종전계약 보증금(만원)']]
    df['계약연월'] = df['계약년월'].astype(str)
    
    df['건축연도대'] = pd.cut(df['건축년도'], bins=[1900, 1989, 1999, 2009, 2019, 2031],
                            labels=['~1989', '1990s', '2000s', '2010s', '2020s'])
    df['면적대'] = pd.cut(df['계약면적(㎡)'], bins=[0, 30, 50, 70, 100, 999],
                         labels=['~30', '30-50', '50-70', '70-100', '100+'])

    # -------------------
    # 2. 지표 계산
    # -------------------
    # 2-1. 보증금 이상치 (Z-score)
    df['조건그룹'] = df[['구', '건축연도대', '면적대']].astype(str).agg('_'.join, axis=1)
    group_stats = df.groupby('조건그룹')['보증금(만원)'].agg(['mean', 'std']).reset_index()
    group_stats.columns = ['조건그룹', '보증금_평균', '보증금_표준편차']
    df = df.merge(group_stats, on='조건그룹', how='left')
    df['보증금_zscore'] = (df['보증금(만원)'] - df['보증금_평균']) / df['보증금_표준편차']
    df['보증금_이상치'] = df['보증금_zscore'].abs() > 2

    # 2-2. 거래 집중 클러스터
    df['도로명_연월'] = df['도로명'] + '_' + df['계약연월']
    addr_month_freq = df['도로명_연월'].value_counts().reset_index()
    addr_month_freq.columns = ['도로명_연월', '주소_해당월_거래수']
    df = df.merge(addr_month_freq, on='도로명_연월', how='left')
    df['거래_집중의심'] = df['주소_해당월_거래수'] >= 3

    # 2-3. 노후 건물 + 이상 보증금
    df['노후건물'] = df['건축년도'] < 1990
    df['노후_이상보증'] = df['노후건물'] & df['보증금_이상치']

    # 2-4. 신규 건물 + 이상 보증금 (신규 건물: 2020년 이후)
    df['신규건물'] = df['건축년도'] >= 2020
    df['신규_이상보증'] = df['신규건물'] & df['보증금_이상치']

    # -------------------
    # 3. 구별 집계
    # -------------------
    구별_지표 = df.groupby('구').agg({
        '보증금_이상치': 'sum',
        '거래_집중의심': 'sum',
        '노후_이상보증': 'sum',
        '신규_이상보증': 'sum',
        '도로명': 'count'
    }).reset_index()

    구별_지표['보증금_이상비율'] = 구별_지표['보증금_이상치'] / 구별_지표['도로명']
    구별_지표['거래집중비율'] = 구별_지표['거래_집중의심'] / 구별_지표['도로명']
    구별_지표['노후이상비율'] = 구별_지표['노후_이상보증'] / 구별_지표['도로명']
    구별_지표['신규이상비율'] = 구별_지표['신규_이상보증'] / 구별_지표['도로명']

    # -------------------
    # 4. 위험스코어 계산
    # -------------------
    score_cols = ['보증금_이상비율', '거래집중비율', '노후이상비율', '신규이상비율']
    scaler = MinMaxScaler()
    구별_지표[[f'{col}_정규화' for col in score_cols]] = scaler.fit_transform(구별_지표[score_cols])
    구별_지표['위험스코어'] = 구별_지표[[f'{col}_정규화' for col in score_cols]].mean(axis=1)

    return 구별_지표.sort_values('위험스코어', ascending=False)